In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
from __future__ import division
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Input, Lambda, Reshape
from keras import backend as K
from keras.optimizers import SGD, Nadam
from keras import layers

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [7]:
from keras.backend.tensorflow_backend import set_session
from utils import limited_gpu_memory_session
set_session(limited_gpu_memory_session())

In [16]:
DATA_DIR = os.path.abspath('./')
CHECKPOINTED_WEIGHTS = os.path.join(DATA_DIR, 'checkpointed_weights.hdf5')
INIT_WEIGHTS = os.path.join(DATA_DIR, 'init_weights_base.hdf5')
EXPERIENCE_BUFFER_FILE = os.path.join(DATA_DIR, 'experience_buffer.p')
MODEL_IMAGE = os.path.join(DATA_DIR, 'siamese_vgg16.png')

In [9]:
from keras.layers.advanced_activations import LeakyReLU
from keras.regularizers import l2, l1

def dense_relu_bn_dropout(x, size, dropout, alpha = 0.1, reg = 0):
    x = Dense(size, kernel_regularizer = l2(reg))(x)
    x = Activation('tanh')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout)(x)
    return x

def create_network(reg, dropout, alpha = 0.1):
    inputs = Input(shape=(INPUT_SHAPE,))
    x = dense_relu_bn_dropout(inputs, 8 , dropout, reg)
    x = dense_relu_bn_dropout(x, 4, dropout, reg)
    x = Dense(1)(x)
    base_network = Model(inputs=inputs, outputs = x)
    print(base_network.summary())
    return base_network

In [10]:
from keras import layers
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, merge, Input, Lambda, Reshape

INPUT_SHAPE = 8

base_network = create_network(reg = 0.5, dropout = 0.5)
input_a = Input(shape=(INPUT_SHAPE,))
processed_a = base_network(input_a)
input_b = Input(shape=(INPUT_SHAPE,))
processed_b = base_network(input_b)
distance = layers.Subtract()([processed_a, processed_b])
out = Activation('sigmoid')(distance)
siamese_net = Model([input_a, input_b], out)
    
siamese_net.save_weights(INIT_WEIGHTS)
print(siamese_net.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
activation_1 (Activation)    (None, 8)                 0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8)                 32        
_________________________________________________________________
dropout_1 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
activation_2 (Activation)    (None, 4)                 0         
__________

In [12]:
import cPickle as pickle
MOVES = pickle.load(open("../moves_dict.p", "rb"))

In [38]:
for key, item in MOVES.iteritems():
    MOVES[key] = np.array(item)

In [46]:
for key in MOVES:
    for x in MOVES[key]:
        if x.shape[0] != 8:
            print(x)
        if 

In [22]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5,
              patience=5, verbose = 1, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_acc',
                              min_delta=1e-4,
                              patience=25,
                              verbose=0, mode='auto')
checkpointer = ModelCheckpoint(filepath=CHECKPOINTED_WEIGHTS, verbose=1, save_best_only=True, monitor='val_acc')

In [24]:
nadam = Nadam(lr=1e-3)
siamese_net.compile(optimizer=nadam, loss='binary_crossentropy', metrics=['accuracy'])
siamese_net.load_weights(INIT_WEIGHTS)

In [ ]:
from utils import DataGenerator

BATCH_SIZE = 512
load_from_file = os.path.exists(EXPERIENCE_BUFFER_FILE)
save_to_file = not load_from_file
datagen = DataGenerator(MOVES, batch_sz = BATCH_SIZE, load_from_file = load_from_file, 
                 save_to_file = save_to_file, file = EXPERIENCE_BUFFER_FILE)

Experience buffer loaded from /home/ubuntu/quackle/rishabh_code/neural_networks/experience_buffer.p
Train: 2272842 Val: 94702


In [ ]:
NUM_TRAIN_PAIRS, NUM_VAL_PAIRS = datagen.get_num_pairs()
STEPS_PER_EPOCH = NUM_TRAIN_PAIRS//BATCH_SIZE
VALIDATION_STEPS = NUM_VAL_PAIRS//BATCH_SIZE
history = siamese_net.fit_generator(
        datagen.next_train(),
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=500,
        validation_data=datagen.next_val(),
        validation_steps=VALIDATION_STEPS,
        callbacks = [reduce_lr, checkpointer, early_stopping])

Epoch 1/500
4439/4439 [==============================] - 29s - loss: 0.6931 - acc: 0.4956 - val_loss: 0.6932 - val_acc: 0.2120
Epoch 2/500
4439/4439 [==============================] - 29s - loss: 0.6931 - acc: 0.4965 - val_loss: 0.6931 - val_acc: 0.2362
Epoch 3/500
4439/4439 [==============================] - 31s - loss: 0.6931 - acc: 0.4941 - val_loss: 0.6932 - val_acc: 0.1735
Epoch 4/500
4439/4439 [==============================] - 29s - loss: 0.6931 - acc: 0.4934 - val_loss: 0.6931 - val_acc: 0.2739
Epoch 5/500
4439/4439 [==============================] - 30s - loss: 0.6931 - acc: 0.4982 - val_loss: 0.6931 - val_acc: 0.2719
Epoch 6/500
4439/4439 [==============================] - 29s - loss: 0.6931 - acc: 0.4974 - val_loss: 0.6931 - val_acc: 0.0734
Epoch 7/500
4439/4439 [==============================] - 30s - loss: 0.6931 - acc: 0.4946 - val_loss: 0.6930 - val_acc: 0.2924
Epoch 8/500
4439/4439 [==============================] - 29s - loss: 0.6931 - acc: 0.4979 - val_loss: 0.6932 - 

4439/4439 [==============================] - 31s - loss: 0.6931 - acc: 0.4973 - val_loss: 0.6931 - val_acc: 0.2592
Epoch 34/500
4438/4439 [============================>.] - ETA: 0s - loss: 0.6931 - acc: 0.4967
Epoch 00033: reducing learning rate to 6.25000029686e-05.
Epoch 00033: val_acc did not improve
4439/4439 [==============================] - 30s - loss: 0.6931 - acc: 0.4967 - val_loss: 0.6931 - val_acc: 0.2609
Epoch 35/500
4439/4439 [==============================] - 30s - loss: 0.6931 - acc: 0.4961 - val_loss: 0.6931 - val_acc: 0.2700
Epoch 36/500
4439/4439 [==============================] - 29s - loss: 0.6931 - acc: 0.4966 - val_loss: 0.6931 - val_acc: 0.2693
Epoch 37/500
4439/4439 [==============================] - 30s - loss: 0.6931 - acc: 0.4961 - val_loss: 0.6931 - val_acc: 0.2325
Epoch 38/500
4439/4439 [==============================] - 29s - loss: 0.6931 - acc: 0.4973 - val_loss: 0.6931 - val_acc: 0.2685
Epoch 39/500
4437/4439 [============================>.] - ETA: 0s - 